## Assignment 8
The goal of this lab is to examine and experiment with several techniques related to influence analysis
in complex networks. Influence is defined between two users and represents how
possible is for a user to adapt the behavior or copy the action of another user. 

We will first introduce the diffusion model which can be used to simulate a spreading process that takes place over the network.
 
Subsequently, we will utilize the structure of the network to identify influencers based in a heuristic manner.

Finally, we will examine the algorithmic solution to the problem of influence maximization,
which is choosing the optimum nodes to maximize the spread of information, and compare it with
the heuristic approach based on the simulations.

We will again rely on the network sciecne dataset for our use case. The following questions can be answered with the help of networkx and NDlib. You may also use other packages to deal with the problem. Please answer the following questions on the networks you have and submit your executable code.

In [1]:
import networkx as nx
from matplotlib import pyplot as plt
import numpy as np
import ndlib.models.epidemics.IndependentCascadesModel as ICM
import ndlib.models.ModelConfig as mc


In [2]:
path = ""

In [3]:
# download a file from a url 
import requests

def download(url,file_name):
    get_response = requests.get(url)
    with open(file_name, "wb") as out_file:
        out_file.write(get_response.content)

download(url = "http://www.casos.cs.cmu.edu/computational_tools/datasets/external/netscience/netscience.gml", file_name=path+"netscience.gml")

In [27]:
# Read the data and show the basic information
graph = nx.read_gml("netscience.gml")

print("Graph Information:")
print(nx.info(graph))


print("\n5 Sample nodes:")
nodes_list = list(graph.nodes(data=True))  # 'data=True' to include attributes

print(nodes_list[:5])

Graph Information:
Graph with 1589 nodes and 2742 edges

5 Sample nodes:
[('ABRAMSON, G', {}), ('KUPERMAN, M', {}), ('ACEBRON, J', {}), ('BONILLA, L', {}), ('PEREZVICENTE, C', {})]


**(a)** We will first create a method to evaluate our chosen seeds, based on epidemic simulation. As a means for evaluation, we use the Independent Cascade model, to compute the number of influenced nodes during an influence spread over the network. 
This model assumes that a node v has only one chance to influence each of its neighbors u based on the probability `p v,u` . 
We define the model's parameters (threshold=0.01) based on common approaches in the literature, and run the epidemic through 10 steps in order to get an approximation and retreive fast results (in normal circumstances it is 10 thousand).


In terms of implementation we will define a function $simulate\_spreading$ that performs the simulation using the IndependentCascadeModel and ModelConfig from NDlib.
More specficially, we have to loop through the edges and add a variable named "threshold" that resembles the probability of influencing a node's neighbors (set to $0.01$ as is common in the literature for dense graphs), using the function $add\_edge\_configuration$.


In [51]:
def simulate_spreading(G, seed_set, sim=10, num_steps=5, threshold=0.01):
    """
    Simulates the influence spread on a graph using the Independent Cascade Model from NDlib.

    Parameters:
    - G (networkx.Graph): Input graph.
    - seed_set (list): List of seed nodes to start the spread.
    - sim (int): Number of simulation runs to average the result.
    - num_steps (int): Number of steps in each simulation.
    - threshold (float): Propagation probability for each edge.

    Returns:
    - float: Average number of influenced nodes after all simulations.
    """
    total_influenced = 0
    
    for _ in range(sim):
        # Initialize the Independent Cascade Model
        model = ICM(G)
        config = mc.Configuration()

        # Add seed nodes to the configuration
        config.add_model_initial_configuration("Infected", seed_set)

        # Add thresholds to edges
        for edge in G.edges():
            config.add_edge_configuration("threshold", edge, threshold)

        # Set the configuration
        model.set_initial_status(config)

        # Simulate for the specified number of steps
        iterations = model.iteration_bunch(num_steps)

        # Track influenced nodes
        influenced_nodes = set()
        for iteration in iterations:
            # Extract nodes with status '1' (infected)
            infected_nodes = [node for node, status in iteration["status"].items() if status == 1]
            influenced_nodes.update(infected_nodes)

        total_influenced += len(influenced_nodes)

    # Return the average number of influenced nodes
    return total_influenced / sim


In [56]:
for i in range(10):
    seed_nodes = list(graph.nodes)[:i+1]  # Select first i+1 nodes 
    total_influenced = simulate_spreading(graph, seed_nodes, sim=10, num_steps=5, threshold=0.1)   # Higher threshold for spread
    print(f"Iteration {i}, Seed Nodes: {seed_nodes}, Total Influenced Nodes: {total_influenced}")

Iteration 0, Seed Nodes: ['ABRAMSON, G'], Total Influenced Nodes: 1.4
Iteration 1, Seed Nodes: ['ABRAMSON, G', 'KUPERMAN, M'], Total Influenced Nodes: 2.4
Iteration 2, Seed Nodes: ['ABRAMSON, G', 'KUPERMAN, M', 'ACEBRON, J'], Total Influenced Nodes: 4.0
Iteration 3, Seed Nodes: ['ABRAMSON, G', 'KUPERMAN, M', 'ACEBRON, J', 'BONILLA, L'], Total Influenced Nodes: 5.4
Iteration 4, Seed Nodes: ['ABRAMSON, G', 'KUPERMAN, M', 'ACEBRON, J', 'BONILLA, L', 'PEREZVICENTE, C'], Total Influenced Nodes: 5.8
Iteration 5, Seed Nodes: ['ABRAMSON, G', 'KUPERMAN, M', 'ACEBRON, J', 'BONILLA, L', 'PEREZVICENTE, C', 'RITORT, F'], Total Influenced Nodes: 6.5
Iteration 6, Seed Nodes: ['ABRAMSON, G', 'KUPERMAN, M', 'ACEBRON, J', 'BONILLA, L', 'PEREZVICENTE, C', 'RITORT, F', 'SPIGLER, R'], Total Influenced Nodes: 7.3
Iteration 7, Seed Nodes: ['ABRAMSON, G', 'KUPERMAN, M', 'ACEBRON, J', 'BONILLA, L', 'PEREZVICENTE, C', 'RITORT, F', 'SPIGLER, R', 'ADAMIC, L'], Total Influenced Nodes: 9.0
Iteration 8, Seed Nodes: 

**(b)** Compute K-core score:  Given our undirected network G, C k is defined as the k-core subgraph of G if it is a maximal connected subgraph in which all nodes have degree at least k. Then, each node v ∈ V has a core number c(v) = k, if it belongs to a k-core but not to a (k + 1)-core. The cohesion of subgraphs increases as k increases. Let us denote as C the set of nodes with the maximum core number k\_max. Compute the top 20 nodes in terms of the k-core they belong to and simulate their spreading.

In [30]:
def compute_kcore(graph):
    core_numbers = nx.core_number(graph)
    
    # Sort nodes based on their core number (highest first)
    sorted_nodes = sorted(core_numbers.items(), key=lambda x: x[1], reverse=True)
    
    # Extract the top 20 nodes based on core number
    top_20_nodes = [node for node, core in sorted_nodes[:20]]
    
    return top_20_nodes

In [32]:
top_20_nodes = compute_kcore(graph)
    
total_influenced = simulate_spreading(graph, top_20_nodes, steps=20, threshold=0.05)
    
print(f"Top 20 nodes: {top_20_nodes}")
print(f"Total Influenced Nodes: {total_influenced}")



Top 20 nodes: ['GIOT, L', 'UETZ, P', 'CAGNEY, G', 'MANSFIELD, T', 'JUDSON, R', 'KNIGHT, J', 'LOCKSHON, D', 'NARAYAN, V', 'SRINIVASAN, M', 'POCHART, P', 'QURESHIEMILI, A', 'LI, Y', 'GODWIN, B', 'CONOVER, D', 'KALBFLEISCH, T', 'VIJAYADAMODAR, G', 'YANG, M', 'JOHNSTON, M', 'FIELDS, S', 'ROTHBERG, J']
Total Influenced Nodes: 20


**(c)** Implement Greedy IM: Influence Maximization is the problem that lies in the heart of influence analysis and addresses how to find a set of nodes, such that if they start a diffusion, the number of infected nodes in the network (influenced spread) will be maximized. It has a broad range of applications, from viral marketing, which was the initial motivation for the problem, to epidemiological containment and political campaign management.
The problem can be formulated as follows: given a social network, a diffusion model with some parameters and a number k, find a seed set S $\subset$ V of size k such that the influence spread is maximized.  We will use the well known method from Kempe et al.[ [1](https://dl.acm.org/doi/pdf/10.1145/956750.956769?casa_token=tkl-2BIJoXIAAAAA:SD8O7LcvEPGyGAdv8cHEwSqgn3Jz0UeHvpRK3-xYB2Z9C7gy-iOQpeHoFqOWzDMfAskBgVzYrpzS)] that is based on the fact that the function of the influence spread under the IC and LT models is monotone non-decreasing and submodular, which gives a  $(1-1/e)$ approximation ratio to the optimal.

To simplify and speed up the implementation to less then a minute, we are going to use only one simple simulation based on the function defined above. Moreover, we will reduce the search space of the algorithm by giving as an input a set of selected nodes to search on. To find a shorter set of such nodes, you can use the filter\_graph function below, that removes nodes under a certain degree, and set an appropriate threshold e.g. 3. We utilize these shortcuts because we need to make the algorithm run in time for the lab. You are encouraged to experiment further in your own time with an implementation that matches more the original algorithm, mainly in order to get a firm understanding of the computational demand of influence maximization.

In [54]:
def filter_graph(G,threshold):
    G_ = G.copy()
    to_remove = [i for i in G_.nodes() if G_.degree(i) < threshold]
    G_.remove_nodes_from(to_remove)
    print("Removed "+str(len(to_remove))+" of "+str(len(G.nodes()))+" nodes")
   
    return G_

In [55]:

def greedy_algorithm(G, selected_nodes, size, sim):
    """
    Greedy influence maximization algorithm for a subset of nodes (Kempe et all at 2003)
    """
   
    

**(d)** Plot the spreading of k-core and greedy IM for a seed set of 20 to compare them.